In [ ]:
import ps3_implementation

## Load data

In [4]:
import scipy.io
mat = scipy.io.loadmat('/Users/yanqingluo/Desktop/LabML/git/problem_set3/data/qm7.mat')
for key, value in mat.items():
    print(f'Key: {key}')
    print() 

Key: __header__

Key: __version__

Key: __globals__

Key: X

Key: R

Key: Z

Key: T

Key: P

